In [6]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-06-03 02:35:41--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2025-06-03 02:35:41 (53.0 MB/s) - ‘minsearch.py’ saved [3832/3832]



# SEARCH DOCs 

In [3]:
import minsearch

In [4]:
import json

In [5]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
for course_dict in docs_raw:
    print(course_dict['course'])

data-engineering-zoomcamp
machine-learning-zoomcamp
mlops-zoomcamp


In [7]:
documents = []

for course_dict in docs_raw:
    for doc_dict in course_dict['documents']:
        doc_dict['course'] = course_dict['course']
        documents.append(doc_dict)

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
# indexing

index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"] 
)
# SELECT * WHERE course="data-engineering-zoomcamp"

In [10]:
index.fit(documents)

In [11]:
q = "the course is already started, can I still enroll?"

In [12]:
boost = {'question': 3.0, 'section': 0.5,}

result = index.search(
    query=q,
    filter_dict = {"course" : "data-engineering-zoomcamp"},
    boost_dict=boost,
    num_results=5
)

In [13]:
result

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

# Using GPT OPENAI API

In [14]:
from openai import OpenAI

client = OpenAI()

In [15]:
q

'the course is already started, can I still enroll?'

In [45]:
response = client.chat.completions.create(
    model = "gpt-4o",
    messages = [{"role": "user", "content": q}] 
)
response

ChatCompletion(id='chatcmpl-BlqtNahRUrqlEJOM1Fvasu6fYiyev', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Whether you can enroll in a course after it has started depends on the institution or platform offering the course, as well as its specific enrollment policies. Here are a few steps you can take:\n\n1. **Check the Course Website or Portal:** Look for any information regarding late enrollments or add/drop deadlines.\n\n2. **Contact the Instructor or Administration:** Reach out directly to the course instructor or the relevant administrative office to inquire about late enrollment possibilities.\n\n3. **Online Platforms:** For courses on platforms like Coursera or edX, check their specific policies on late enrollments, as some online courses offer flexible start dates or self-paced options.\n\n4. **Consider Future Offerings:** If late enrollment isn't possible, ask if the course will be offered again in the near future and what y

In [19]:
print(response.choices[0].message.content)

Whether you can still enroll in a course that has already started depends on several factors, including the policies of the institution offering the course and the specific course requirements. Here are a few steps you can take:

1. **Contact the Instructor or Institution**: Reach out directly to the course instructor or the admissions office to inquire about the possibility of late enrollment.
   
2. **Check Enrollment Policies**: Review the institution’s enrollment policies, which are often available on their website. Some institutions allow late enrollment with certain conditions.

3. **Assess the Course Format**: If the course is self-paced or online, you might have more flexibility in joining after the start date.

4. **Catch-Up Plan**: If late enrollment is possible, ask what materials you will need and how you can catch up on missed content to stay on track.

If you get approval to join the course late, be prepared to put in extra work initially to catch up.


# Applying Prompt Template for FAQ 

In [38]:
prompt_template = """
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not contain answer, output NONE.

QUESTION : {question}
CONTEXT : 
{context}
"""

In [28]:
context = ""

for doc in result:
    context = context + f"section : {doc['section']}\nquestion : {doc['question']}\nanswer : {doc['text']}\n\n"

In [29]:
print(context)

section : General course-related questions
question : Course - Can I still join the course after the start date?
answer : Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section : General course-related questions
question : Course - Can I follow the course after it finishes?
answer : Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section : General course-related questions
question : Course - When will the course start?
answer : The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will 

In [34]:
prompt_template

'\nYou are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\nIf the CONTEXT does not contain answer, output NONE.\n\nQUESTION : {question}\nCONTEXT : {context}\n'

In [42]:
prompt = prompt_template.format(question=q, context=context).strip()
print(prompt)

You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not contain answer, output NONE.

QUESTION : the course is already started, can I still enroll?
CONTEXT : 
section : General course-related questions
question : Course - Can I still join the course after the start date?
answer : Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section : General course-related questions
question : Course - Can I follow the course after it finishes?
answer : Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on yo

In [46]:
response = client.chat.completions.create(
    model = "gpt-4o",
    messages = [{"role": "user", "content": prompt}]
)
response.choices[0].message.content

'Yes, you can still enroll in the course after it has started. You are eligible to submit homework, but be aware of the deadlines for the final projects.'